In [1]:
import numpy as np
import pandas as pd

In [2]:
tms_acutal_dir = "../../data/actual/TMS_Actual.csv"
flow_acutal_dir = "../../data/actual/FLOW_Actual.csv"
aws368_dir = "../../data/actual/AWS_368.csv"
aws541_dir = "../../data/actual/AWS_541.csv"
aws569_dir = "../../data/actual/AWS_569.csv"

In [3]:
tms = pd.read_csv(tms_acutal_dir)
flow = pd.read_csv(flow_acutal_dir)
aws368 = pd.read_csv(aws368_dir)
aws541 = pd.read_csv(aws541_dir)
aws569 = pd.read_csv(aws569_dir)

In [4]:
tms['SYS_TIME'] = pd.to_datetime(tms['SYS_TIME'])
flow['SYS_TIME'] = pd.to_datetime(flow['SYS_TIME'])
aws368['datetime'] = pd.to_datetime(aws368['datetime'])
aws541['datetime'] = pd.to_datetime(aws541['datetime'])
aws569['datetime'] = pd.to_datetime(aws569['datetime'])

In [5]:
tms = tms.set_index('SYS_TIME').interpolate(method="linear")
flow = flow.set_index('SYS_TIME').interpolate(method="linear")
aws368 = aws368.set_index('datetime').interpolate(method="linear")
aws541 = aws541.set_index('datetime').interpolate(method="linear")
aws569 = aws569.set_index('datetime').interpolate(method="linear")

In [88]:
tms.info()

<class 'pandas.DataFrame'>
DatetimeIndex: 514551 entries, 2024-08-26 15:09:00 to 2025-09-29 05:23:00
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   TOC_VU   514551 non-null  float64
 1   PH_VU    514551 non-null  float64
 2   SS_VU    514551 non-null  float64
 3   FLUX_VU  514551 non-null  float64
 4   TN_VU    514551 non-null  float64
 5   TP_VU    514551 non-null  float64
dtypes: float64(6)
memory usage: 27.5 MB


In [6]:
# 컬럼명 변경
aws368 = aws368.add_suffix('_368')
aws541 = aws541.add_suffix('_541')
aws569 = aws569.add_suffix('_569')

# join
flow_df = flow.join([aws368, aws541, aws569], how='left')
tms_df = tms.join([aws368, aws541, aws569], how='left')
all_df = flow.join([tms, aws368, aws541, aws569], how = "inner")

In [7]:
# 특정 패턴 포함 컬럼 삭제
cols_to_drop = [col for col in flow_df.columns if col in ['data_save_dt', 'YYMMDDHHMI_368', 'YYMMDDHHMI_541', 'YYMMDDHHMI_569', 'STN_368', 'STN_541', 'STN_569']]
flow_df = flow_df.drop(columns=cols_to_drop)

# 특정 패턴 포함 컬럼 삭제
cols_to_drop = [col for col in tms_df.columns if col in ['data_save_dt', 'YYMMDDHHMI_368', 'YYMMDDHHMI_541', 'YYMMDDHHMI_569', 'STN_368', 'STN_541', 'STN_569']]
tms_df = tms_df.drop(columns=cols_to_drop)

# 특정 패턴 포함 컬럼 삭제
cols_to_drop = [col for col in all_df.columns if col in ['data_save_dt', 'YYMMDDHHMI_368', 'YYMMDDHHMI_541', 'YYMMDDHHMI_569', 'STN_368', 'STN_541', 'STN_569']]
all_df = all_df.drop(columns=cols_to_drop)

In [93]:
flow_df

,flow_TankA,flow_TankB,level_TankA,level_TankB,TA_368,RN_15m_368,RN_60m_368,RN_12H_368,RN_DAY_368,HM_368,...,RN_DAY_541,HM_541,TD_541,TA_569,RN_15m_569,RN_60m_569,RN_12H_569,RN_DAY_569,HM_569,TD_569
SYS_TIME,,,,,,,,,,,,,,,,,,,,,
2025-09-02 23:53:00,230.4000,230.2500,3.55225,3.54900,23.5,0.0,0.0,0.0,0.0,83.9,...,0.0,92.7,21.6,22.9,0.0,0.0,0.0,0.0,86.7,20.6
2025-09-02 23:54:00,229.9875,230.1000,3.54700,3.54575,23.5,0.0,0.0,0.0,0.0,84.0,...,0.0,92.6,21.5,22.9,0.0,0.0,0.0,0.0,87.2,20.7
2025-09-02 23:55:00,229.5000,230.0625,3.54825,3.54475,23.5,0.0,0.0,0.0,0.0,84.1,...,0.0,92.6,21.5,22.9,0.0,0.0,0.0,0.0,87.3,20.7
2025-09-02 23:56:00,229.8375,230.4750,3.54125,3.54250,23.5,0.0,0.0,0.0,0.0,84.1,...,0.0,92.6,21.5,22.9,0.0,0.0,0.0,0.0,86.8,20.6
2025-09-02 23:57:00,229.9125,230.0250,3.54050,3.54050,23.5,0.0,0.0,0.0,0.0,84.1,...,0.0,92.5,21.5,23.0,0.0,0.0,0.0,0.0,86.9,20.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-03 10:35:00,199.8000,200.6250,2.43450,2.42275,-6.1,0.0,0.0,0.0,0.0,25.3,...,0.0,26.2,-21.7,-5.0,0.0,0.0,0.0,0.0,24.3,-22.3
2025-12-03 10:36:00,199.0125,200.1375,2.43200,2.42350,-6.1,0.0,0.0,0.0,0.0,25.6,...,0.0,26.4,-21.5,-4.9,0.0,0.0,0.0,0.0,24.0,-22.4
2025-12-03 10:37:00,199.5750,199.3875,2.42975,2.42650,-6.0,0.0,0.0,0.0,0.0,25.7,...,0.0,26.5,-21.4,-4.7,0.0,0.0,0.0,0.0,24.1,-22.2


In [95]:
tms_df.info()

<class 'pandas.DataFrame'>
DatetimeIndex: 514551 entries, 2024-08-26 15:09:00 to 2025-09-29 05:23:00
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   TOC_VU      514551 non-null  float64
 1   PH_VU       514551 non-null  float64
 2   SS_VU       514551 non-null  float64
 3   FLUX_VU     514551 non-null  float64
 4   TN_VU       514551 non-null  float64
 5   TP_VU       514551 non-null  float64
 6   TA_368      514551 non-null  float64
 7   RN_15m_368  514551 non-null  float64
 8   RN_60m_368  514551 non-null  float64
 9   RN_12H_368  514551 non-null  float64
 10  RN_DAY_368  514551 non-null  float64
 11  HM_368      514551 non-null  float64
 12  TD_368      514551 non-null  float64
 13  TA_541      514551 non-null  float64
 14  RN_15m_541  514551 non-null  float64
 15  RN_60m_541  514551 non-null  float64
 16  RN_12H_541  514551 non-null  float64
 17  RN_DAY_541  514551 non-null  float64
 18  HM_541      514551 no

In [97]:
all_df.info()

<class 'pandas.DataFrame'>
DatetimeIndex: 37486 entries, 2025-09-02 23:53:00 to 2025-09-29 05:23:00
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   flow_TankA   37486 non-null  float64
 1   flow_TankB   37486 non-null  float64
 2   level_TankA  37486 non-null  float64
 3   level_TankB  37486 non-null  float64
 4   TOC_VU       37486 non-null  float64
 5   PH_VU        37486 non-null  float64
 6   SS_VU        37486 non-null  float64
 7   FLUX_VU      37486 non-null  float64
 8   TN_VU        37486 non-null  float64
 9   TP_VU        37486 non-null  float64
 10  TA_368       37486 non-null  float64
 11  RN_15m_368   37486 non-null  float64
 12  RN_60m_368   37486 non-null  float64
 13  RN_12H_368   37486 non-null  float64
 14  RN_DAY_368   37486 non-null  float64
 15  HM_368       37486 non-null  float64
 16  TD_368       37486 non-null  float64
 17  TA_541       37486 non-null  float64
 18  RN_15m_541   37486 non

In [8]:
flow_df.to_csv('../../data/processed/flow_proc.csv', date_format='%Y-%m-%d %H:%M:%S', index=True)
tms_df.to_csv('../../data/processed/tms_proc.csv', date_format='%Y-%m-%d %H:%M:%S', index=True)
all_df.to_csv('../../data/processed/all_proc.csv', date_format='%Y-%m-%d %H:%M:%S', index=True)

In [99]:
test = pd.read_csv('../../data/processed/flow_proc.csv', 
                   parse_dates=[0], 
                   index_col=0)
print(test.head())

                     flow_TankA  flow_TankB  level_TankA  level_TankB  TA_368  \
SYS_TIME                                                                        
2025-09-02 23:53:00    230.4000    230.2500      3.55225      3.54900    23.5   
2025-09-02 23:54:00    229.9875    230.1000      3.54700      3.54575    23.5   
2025-09-02 23:55:00    229.5000    230.0625      3.54825      3.54475    23.5   
2025-09-02 23:56:00    229.8375    230.4750      3.54125      3.54250    23.5   
2025-09-02 23:57:00    229.9125    230.0250      3.54050      3.54050    23.5   

                     RN_15m_368  RN_60m_368  RN_12H_368  RN_DAY_368  HM_368  \
SYS_TIME                                                                      
2025-09-02 23:53:00         0.0         0.0         0.0         0.0    83.9   
2025-09-02 23:54:00         0.0         0.0         0.0         0.0    84.0   
2025-09-02 23:55:00         0.0         0.0         0.0         0.0    84.1   
2025-09-02 23:56:00         0.0      